# HW4

In [ ]:
!wget https://medicalanalytics.group/wp-content/uploads/2020/04/dataonly.zip

--2024-10-09 20:38:39--  https://medicalanalytics.group/wp-content/uploads/2020/04/dataonly.zip
Resolving medicalanalytics.group (medicalanalytics.group)... 192.0.78.25, 192.0.78.24
Connecting to medicalanalytics.group (medicalanalytics.group)|192.0.78.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53884239 (51M) [application/zip]
Saving to: ‘dataonly.zip’

dataonly.zip        100%[===================>]  51.39M  61.4MB/s    in 0.8s    

2024-10-09 20:38:40 (61.4 MB/s) - ‘dataonly.zip’ saved [53884239/53884239]



In [ ]:
!unzip dataonly.zip -d dataonly

Archive:  dataonly.zip
  inflating: dataonly/DataOnly/WaitData.Published.xlsx  


In [ ]:
import pandas as pd

In [ ]:
FILEPATH = "./dataonly/DataOnly/WaitData.Published.xlsx"
os.path.isfile(FILEPATH)

True

In [ ]:
df = pd.read_excel(FILEPATH, sheet_name='F3')

In [ ]:
df = df.drop(columns=[col for col in df.columns if col.startswith('x_')])

In [ ]:
df.head()

,Wait,SumHowEarlyWaiting,AvgHowEarlyWaiting,LineCount0Strict,SumWaits,LineCount0,LineCount1,LineCount2,LineCount3,LineCount4,...,NumAddOnsToday,NumAddOnsLastW2,NumScheduledNextSlot,NumScheduledNextW2,SumTimeToCompleteNextSlot,SumTimeToCompleteNextW2,WithContrastCountWaiting,WithAndWithoutContrastCountWaiting,WithContrastCountInProgress,WithAndWithoutContrastCountInProgress
0,0,372,62,6,249,6,5,6,6,4,...,4,2,0,0,0,0,3,1,1,0
1,-3,335,67,3,195,5,4,6,6,5,...,4,1,2,0,32,0,2,1,2,0
2,-25,274,68,4,150,4,5,6,4,6,...,4,1,1,1,13,16,1,1,3,0
3,36,369,73,2,179,5,6,4,5,5,...,3,1,2,1,45,16,3,1,1,0
4,-21,438,73,5,192,6,5,5,6,5,...,3,0,2,1,32,15,4,0,0,1


In [ ]:
X, Y = df.drop(columns=['Wait']), df.Wait

In [ ]:
from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(X, Y)
Ypred = model.predict(X) # use trained regression model to predict
r = Y-Ypred # compute prediatin error (residual)
e=abs(r).mean() # compute model error

In [ ]:
e

23.16628893634719

In [ ]:
abs(r).median()

19.126063446073886

In [ ]:
# Run Python feature selection
if True: # just in case I want to disable this part
    print('\n>Python feature selection:')
    from sklearn.feature_selection import RFE
    from itertools import compress
    for nFeatures in range(1,4):
        rfe = RFE(model, n_features_to_select=nFeatures)
        X_rfe = rfe.fit_transform(X,Y) #transforming data using RFE
        #Fitting the data to model
        model.fit(X_rfe,Y)
        #print(rfe.support_)
        #print(rfe.ranking_)
        cols = list(compress( X.columns, rfe.support_))
        model.fit(X[cols],Y)
        e = abs(Y-model.predict(X[cols])).mean()
        print(e, cols)


>Python feature selection:
32.50541172120655 ['CardiacCount']
31.22493770698274 ['CardiacCount', 'DelayedInLine']
31.25752894868932 ['CardiacCount', 'StartTime', 'DelayedInLine']


In [ ]:
X.columns.shape[0]

85

In [ ]:
best_feature = None
min_error = float('inf')

for feature in X.columns:
    model.fit(X[[feature]], Y)
    e = abs(Y - model.predict(X[[feature]])).mean()

    if e < min_error:
        min_error = e
        best_feature = feature

min_error, best_feature

(31.227057699733514, 'DelayedInLine')

In [ ]:
best_features = list()
min_errors = list()

for i in range(3):
    current_best = None
    current_error = float('inf')
    for feature in X.columns:
        if feature not in best_features:
            _features = best_features + [feature]
            model.fit(X[_features], Y)
            e = abs(Y - model.predict(X[_features])).mean()

            if e < current_error:
                current_error = e
                current_best = feature

    best_features.append(current_best)
    min_errors.append(current_error)

min_errors, best_features

([31.227057699733514, 30.137965863467567, 28.696490632253404],
 ['DelayedInLine', 'LineCount0Strict', 'LineCount0'])

In [ ]:
best_features = list()
min_errors = list()

for i in range(15):
    current_best = None
    current_error = float('inf')
    for feature in X.columns:
        if feature not in best_features:
            _features = best_features + [feature]
            model.fit(X[_features], Y)
            e = abs(Y - model.predict(X[_features])).mean()

            if e < current_error:
                current_error = e
                current_best = feature

    best_features.append(current_best)
    min_errors.append(current_error)

min_errors, best_features

([31.227057699733514,
  30.137965863467567,
  28.696490632253404,
  28.246309585132344,
  28.147006842164537,
  24.989574734774788,
  24.65918640725134,
  24.329071006187295,
  24.071936786758204,
  23.846509492065334,
  23.69261939854711,
  23.627914296582187,
  23.57915967047997,
  23.548081793881497,
  23.520963512553436],
 ['DelayedInLine',
  'LineCount0Strict',
  'LineCount0',
  'SumDelayWaitingInLine',
  'AheadCount',
  'NumCompletedToday',
  'NumScheduledNextW2',
  'DelayCount',
  'InProgressSize',
  'StartTime4',
  'AvgDelayForDay',
  'AvgWaitLastK1Customers',
  'SumDelayInProgress',
  'NumScheduledNextSlot',
  'IsFirst'])

In [ ]:
for i in range(1, len(min_errors) + 1):
    print(f"{i}\t{min_errors[i - 1]}")

1	31.227057699733514
2	30.137965863467567
3	28.696490632253404
4	28.246309585132344
5	28.147006842164537
6	24.989574734774788
7	24.65918640725134
8	24.329071006187295
9	24.071936786758204
10	23.846509492065334
11	23.69261939854711
12	23.627914296582187
13	23.57915967047997
14	23.548081793881497
15	23.520963512553436


In [ ]:
min_error = float('inf')
best_features = None

for i in range(len(X.columns)):
    for j in range(i + 1, len(X.columns)):
        for k in range(j + 1, len(X.columns)):
            f1 = X.columns[i]
            f2 = X.columns[j]
            f3 = X.columns[k]
            model = linear_model.LinearRegression()
            model.fit(X[[f1, f2, f3]], Y)
            e = abs(Y - model.predict(X[[f1, f2, f3]])).mean()

            if e < min_error:
                min_error = e
                best_features = (f1, f2, f3)

In [ ]:
min_error, best_features

(26.072745167091224,
 ('AheadCount', 'SumDelayWaitingByExamCode', 'NumCompletedToday'))